#Analyse des Tendances Musicales sur Spotify

##Importance du projet


Ce projet est crucial car il permet de :

Soutenir les artistes : En leur fournissant des insights sur les préférences des auditeurs.

Informer les professionnels de l'industrie musicale : Qui peuvent utiliser ces données pour orienter leurs choix marketing et de production.

Contribuer à la recherche académique : Sur l'impact des caractéristiques musicales sur la popularité.

##Objectif du projet

L'objectif de notre projet est d'analyser un ensemble de données provenant de Spotify pour :

Identifier les tendances musicales : Comprendre quelles caractéristiques font un morceau populaire.
Explorer les relations entre différentes caractéristiques : Par exemple, comment l'énergie ou la danseabilité influence la popularité d'une piste.
Proposer des recommandations : Utiliser les résultats pour aider les artistes à créer des morceaux qui résonnent avec leur public.

##Données utilisées

Nous avons utilisé un ensemble de données contenant des informations sur des morceaux de musique, y compris :

Identifiants des morceaux et des artistes
Caractéristiques acoustiques : danseabilité, énergie, tonalité, etc.
Popularité des morceaux : mesurée par le nombre d'écoutes.
Langue des morceaux et autres métadonnées

##Méthodologie

Préparation des Données : Nous avons sélectionné les colonnes pertinentes pour notre analyse.

Analyse Descriptive : Nous avons généré des statistiques descriptives pour comprendre la distribution des caractéristiques des morceaux.

Classification des Caractéristiques : Nous avons créé des descriptions basées sur des seuils pour des caractéristiques comme la danseabilité, l'énergie et la popularité. Cela nous aide à catégoriser les morceaux selon leur ambiance.

Filtrage des Morceaux Populaires : Nous avons appliqué des filtres pour identifier les morceaux ayant une popularité, ce qui nous permet de nous concentrer sur les pistes les plus pertinentes.

##Identification des Tendances Musicales via l'Analyse des Associations

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
spotify_data = pd.read_csv('./spotify_tracks.csv', sep=',')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
spotify_data.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(62317, 22)

In [4]:
# Afficher les colonnes du DataFrame
spotify_data.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['track_id', 'track_name', 'artist_name', 'year', 'popularity',
       'artwork_url', 'album_name', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence',
       'track_url', 'language'],
      dtype='object')

In [5]:
# Sélectionner les colonnes d'intérêt
key_features = spotify_data[['track_id', 'track_name', 'artist_name', 'year', 'popularity',
                              'album_name', 'acousticness', 'danceability', 'duration_ms',
                              'energy', 'key', 'liveness', 'loudness', 'mode',
                              'speechiness', 'tempo', 'time_signature', 'valence',
                              'track_url', 'language']]
key_features.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,track_id,track_name,artist_name,year,popularity,album_name,acousticness,danceability,duration_ms,energy,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,track_url,language
0,2r0ROhr7pRN4MXDMT1fEmd,"Leo Das Entry (From ""Leo"")",Anirudh Ravichander,2024,59,"Leo Das Entry (From ""Leo"")",0.0241,0.753,97297.0,0.970,8.0,0.1000,-5.994,0.0,0.1030,110.997,4.0,0.459,https://open.spotify.com/track/2r0ROhr7pRN4MXD...,Tamil
1,4I38e6Dg52a2o2a8i5Q5PW,AAO KILLELLE,"Anirudh Ravichander, Pravin Mani, Vaishali Sri...",2024,47,AAO KILLELLE,0.0851,0.780,207369.0,0.793,10.0,0.0951,-5.674,0.0,0.0952,164.995,3.0,0.821,https://open.spotify.com/track/4I38e6Dg52a2o2a...,Tamil
2,59NoiRhnom3lTeRFaBzOev,Mayakiriye Sirikiriye - Orchestral EDM,"Anirudh Ravichander, Anivee, Alvin Bruno",2024,35,Mayakiriye Sirikiriye (Orchestral EDM),0.0311,0.457,82551.0,0.491,2.0,0.0831,-8.937,0.0,0.1530,169.996,4.0,0.598,https://open.spotify.com/track/59NoiRhnom3lTeR...,Tamil
3,5uUqRQd385pvLxC8JX3tXn,Scene Ah Scene Ah - Experimental EDM Mix,"Anirudh Ravichander, Bharath Sankar, Kabilan, ...",2024,24,Scene Ah Scene Ah (Experimental EDM Mix),0.2270,0.718,115831.0,0.630,7.0,0.1240,-11.104,1.0,0.4450,169.996,4.0,0.362,https://open.spotify.com/track/5uUqRQd385pvLxC...,Tamil
4,1KaBRg2xgNeCljmyxBH1mo,Gundellonaa X I Am A Disco Dancer - Mashup,"Anirudh Ravichander, Benny Dayal, Leon James, ...",2024,22,Gundellonaa X I Am a Disco Dancer (Mashup),0.0153,0.689,129621.0,0.748,7.0,0.3450,-9.637,1.0,0.1580,128.961,4.0,0.593,https://open.spotify.com/track/1KaBRg2xgNeCljm...,Tamil


In [6]:
# Obtenir des statistiques descriptives
spotify_data.describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,year,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,62317.000000,62317.000000,62317.000000,62317.000000,6.231700e+04,62317.000000,62317.000000,62317.000000,62317.000000,62317.000000,62317.000000,62317.000000,62317.000000,62317.000000,62317.000000
mean,2014.425935,15.358361,0.362292,0.596807,2.425270e+05,0.602496,0.146215,5.101658,0.194143,-65.103433,0.586052,0.087722,117.931247,3.857086,0.495226
std,9.645113,18.626908,0.314609,0.186209,1.129999e+05,0.246144,0.307804,3.553469,0.172030,2369.051478,0.493682,0.115150,28.509459,0.502660,0.264787
min,1971.000000,0.000000,-1.000000,-1.000000,5.000000e+03,-1.000000,-1.000000,-1.000000,-1.000000,-100000.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,2011.000000,0.000000,0.067100,0.497000,1.921600e+05,0.440000,0.000000,2.000000,0.093200,-10.727000,0.000000,0.036700,95.942000,4.000000,0.292000
50%,2017.000000,7.000000,0.286000,0.631000,2.362670e+05,0.639000,0.000025,5.000000,0.125000,-7.506000,1.000000,0.048900,117.991000,4.000000,0.507000
75%,2022.000000,26.000000,0.632000,0.730000,2.862400e+05,0.803000,0.015200,8.000000,0.243000,-5.456000,1.000000,0.089100,135.081000,4.000000,0.710000
max,2024.000000,93.000000,0.996000,0.986000,4.581483e+06,1.000000,0.999000,11.000000,0.998000,1.233000,1.000000,0.959000,239.970000,5.000000,0.995000


In [7]:
# Pour mettre en avant les pistes musicales les plus populaires.
most_popular_tracks = spotify_data.sort_values('popularity', ascending=False).head(10)
# Pour les visualiser
most_popular_tracks[['track_name', 'artist_name', 'popularity']]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,track_name,artist_name,popularity
49479,Big Dawgs,"Hanumankind, Kalmi",93
26580,Starboy,"The Weeknd, Daft Punk",91
26432,Blinding Lights,The Weeknd,91
54962,Bye Bye Bye - From Deadpool and Wolverine Soun...,*NSYNC,90
24273,cardigan,Taylor Swift,89
23702,Anti-Hero,Taylor Swift,89
23514,Is It Over Now? (Taylor's Version) (From The V...,Taylor Swift,88
24274,august,Taylor Swift,88
31241,EASY,LE SSERAFIM,88
38114,Locked out of Heaven,Bruno Mars,88


In [8]:
# Afficher les langues comme index et le nombre d'occurrences comme valeurs
language_counts = spotify_data['language'].value_counts()
language_counts

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,count
language,
English,23392
Unknown,13005
Tamil,12681
Korean,6893
Hindi,5740
Telugu,324
Malayalam,282


In [9]:
# Sélectionner les colonnes pertinentes
features_of_interest = ['track_name', 'danceability', 'energy',
                        'acousticness', 'valence', 'loudness', 'tempo', 'language','popularity']

# Créer un DataFrame avec les caractéristiques
features_df = spotify_data[features_of_interest]

# Définir des descriptions basées sur des seuils
def describe_danceability(value):
    if value < 0.4:
        return "danceability: Faible"
    elif value < 0.7:
        return "danceability: Moyenne"
    else:
        return "danceability: Élevée"

def describe_energy(value):
    if value < 0.4:
        return "energy: Faible"
    elif value < 0.7:
        return "energy: Moyenne"
    else:
        return "energy: Élevée"

def describe_acousticness(value):
    if value < 0.5:
        return "acousticness: Énergique"
    else:
        return "acousticness: Calme"

def describe_valence(value):
    if value < 0.5:
        return "valence: Sombre"
    else:
        return "valence: Joyeux"

def describe_loudness(value):
    if value < -10:
        return "loudness: Faible"
    elif value < -5:
        return "loudness: Moyenne"
    else:
        return "loudness: Élevée"

def describe_tempo(value):
    if value < 100:
        return "tempo: Lent"
    elif value < 120:
        return "tempo: Moyen"
    else:
        return "tempo: Rapide"

def describe_language(value):
    languages = {
        'Tamil': "language: Tamil",
        'English': "language: English",
        'Hindi': "language: Hindi",
        'Korean': "language: Korean",
        'Telugu': "language: Telugu",
        'Malayalam': "language: Malayalam",
        'Unknown': "language: Unknown"
    }
    return languages.get(value, "language: Autre")

def describe_popularity(value):
    if value < 20:
        return "popularité: Faible"
    elif value < 40:
        return "popularité: Moyenne"
    else:
        return "popularité: Élevée"

# Appliquer les descriptions
features_df['danceability'] = features_df['danceability'].apply(describe_danceability)
features_df['energy'] = features_df['energy'].apply(describe_energy)
features_df['acousticness'] = features_df['acousticness'].apply(describe_acousticness)
features_df['valence'] = features_df['valence'].apply(describe_valence)
features_df['loudness'] = features_df['loudness'].apply(describe_loudness)
features_df['tempo'] = features_df['tempo'].apply(describe_tempo)
features_df['language'] = features_df['language'].apply(describe_language)
features_df['popularity'] = features_df['popularity'].apply(describe_popularity)

# Afficher le nouveau DataFrame avec track_name
features_df.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-9-a89b53a6c551>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df['danceability'] = features_df['danceability'].apply(describe_danceability)
<ipython-input-9-a89b53a6c551>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

,track_name,danceability,energy,acousticness,valence,loudness,tempo,language,popularity
0,"Leo Das Entry (From ""Leo"")",danceability: Élevée,energy: Élevée,acousticness: Énergique,valence: Sombre,loudness: Moyenne,tempo: Moyen,language: Tamil,popularité: Élevée
1,AAO KILLELLE,danceability: Élevée,energy: Élevée,acousticness: Énergique,valence: Joyeux,loudness: Moyenne,tempo: Rapide,language: Tamil,popularité: Élevée
2,Mayakiriye Sirikiriye - Orchestral EDM,danceability: Moyenne,energy: Moyenne,acousticness: Énergique,valence: Joyeux,loudness: Moyenne,tempo: Rapide,language: Tamil,popularité: Moyenne
3,Scene Ah Scene Ah - Experimental EDM Mix,danceability: Élevée,energy: Moyenne,acousticness: Énergique,valence: Sombre,loudness: Faible,tempo: Rapide,language: Tamil,popularité: Moyenne
4,Gundellonaa X I Am A Disco Dancer - Mashup,danceability: Moyenne,energy: Élevée,acousticness: Énergique,valence: Joyeux,loudness: Moyenne,tempo: Rapide,language: Tamil,popularité: Moyenne
5,"Villain Yevadu Ra (From ""Leo (Telugu)"")",danceability: Moyenne,energy: Élevée,acousticness: Énergique,valence: Sombre,loudness: Moyenne,tempo: Moyen,language: Telugu,popularité: Moyenne
6,Gundellonaa - Pop Kuthu,danceability: Moyenne,energy: Moyenne,acousticness: Énergique,valence: Joyeux,loudness: Moyenne,tempo: Rapide,language: Tamil,popularité: Faible
7,Knockout Song - Funk Mix,danceability: Élevée,energy: Moyenne,acousticness: Énergique,valence: Joyeux,loudness: Moyenne,tempo: Moyen,language: English,popularité: Faible
8,Rathamaarey (LoFi Version),danceability: Moyenne,energy: Moyenne,acousticness: Énergique,valence: Sombre,loudness: Moyenne,tempo: Rapide,language: Tamil,popularité: Moyenne
9,Theeraamal - Ambient Mix,danceability: Moyenne,energy: Moyenne,acousticness: Calme,valence: Sombre,loudness: Moyenne,tempo: Rapide,language: Tamil,popularité: Faible


In [10]:
# Pour accéder facilement aux données sous forme de tableau.
list_final = features_df.values

# Pour convertir le tableau en une liste de listes
records_list = list_final.tolist()

# La nouvelle liste contenant toutes les listes d'enregistrements sans le nom de la piste
without_track_name = [record[1:] for record in records_list]

# Afficher la liste complète
without_track_name[:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['danceability: Élevée',
  'energy: Élevée',
  'acousticness: Énergique',
  'valence: Sombre',
  'loudness: Moyenne',
  'tempo: Moyen',
  'language: Tamil',
  'popularité: Élevée'],
 ['danceability: Élevée',
  'energy: Élevée',
  'acousticness: Énergique',
  'valence: Joyeux',
  'loudness: Moyenne',
  'tempo: Rapide',
  'language: Tamil',
  'popularité: Élevée'],
 ['danceability: Moyenne',
  'energy: Moyenne',
  'acousticness: Énergique',
  'valence: Joyeux',
  'loudness: Moyenne',
  'tempo: Rapide',
  'language: Tamil',
  'popularité: Moyenne'],
 ['danceability: Élevée',
  'energy: Moyenne',
  'acousticness: Énergique',
  'valence: Sombre',
  'loudness: Faible',
  'tempo: Rapide',
  'language: Tamil',
  'popularité: Moyenne'],
 ['danceability: Moyenne',
  'energy: Élevée',
  'acousticness: Énergique',
  'valence: Joyeux',
  'loudness: Moyenne',
  'tempo: Rapide',
  'language: Tamil',
  'popularité: Moyenne']]

In [11]:
# Pour se concentrer sur les morceaux les plus populaires
hpt = features_df[features_df['popularity'] == "popularité: Élevée"]

hpt.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,track_name,danceability,energy,acousticness,valence,loudness,tempo,language,popularity
0,"Leo Das Entry (From ""Leo"")",danceability: Élevée,energy: Élevée,acousticness: Énergique,valence: Sombre,loudness: Moyenne,tempo: Moyen,language: Tamil,popularité: Élevée
1,AAO KILLELLE,danceability: Élevée,energy: Élevée,acousticness: Énergique,valence: Joyeux,loudness: Moyenne,tempo: Rapide,language: Tamil,popularité: Élevée
38,Chaleya,danceability: Moyenne,energy: Élevée,acousticness: Énergique,valence: Sombre,loudness: Moyenne,tempo: Lent,language: Tamil,popularité: Élevée
39,"Chaleya (From ""Jawan"")",danceability: Moyenne,energy: Élevée,acousticness: Énergique,valence: Sombre,loudness: Moyenne,tempo: Lent,language: Tamil,popularité: Élevée
40,"Ordinary Person (From ""Leo"")",danceability: Élevée,energy: Élevée,acousticness: Énergique,valence: Joyeux,loudness: Moyenne,tempo: Lent,language: Tamil,popularité: Élevée
41,Hukum - Thalaivar Alappara,danceability: Élevée,energy: Élevée,acousticness: Énergique,valence: Joyeux,loudness: Moyenne,tempo: Moyen,language: Tamil,popularité: Élevée
42,"Hayyoda (From ""Jawan"")",danceability: Moyenne,energy: Élevée,acousticness: Énergique,valence: Sombre,loudness: Moyenne,tempo: Lent,language: Tamil,popularité: Élevée
43,"Naa Ready (From ""Leo"")",danceability: Élevée,energy: Élevée,acousticness: Énergique,valence: Joyeux,loudness: Moyenne,tempo: Lent,language: Tamil,popularité: Élevée
44,"Badass (From ""Leo"")",danceability: Moyenne,energy: Élevée,acousticness: Énergique,valence: Joyeux,loudness: Moyenne,tempo: Lent,language: Tamil,popularité: Élevée
45,Not Ramaiya Vastavaiya,danceability: Moyenne,energy: Élevée,acousticness: Énergique,valence: Sombre,loudness: Moyenne,tempo: Rapide,language: Tamil,popularité: Élevée


In [12]:
# Pour créer un encodeur de transactions pour convertir les données en format binaire.
transactions = TransactionEncoder()

# Pour ajuster l'encodeur aux données de 'without_track_name' et transformer ces données en un tableau binaire.
transactions_array = transactions.fit(without_track_name).transform(without_track_name)

# Pour convertir le tableau binaire en un DataFrame avec les noms de colonnes appropriés.
df_train = pd.DataFrame(transactions_array, columns=transactions.columns_)

# Pour afficher les 3 premières lignes du DataFrame résultant.
df_train.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,acousticness: Calme,acousticness: Énergique,danceability: Faible,danceability: Moyenne,danceability: Élevée,energy: Faible,energy: Moyenne,energy: Élevée,language: English,language: Hindi,...,loudness: Moyenne,loudness: Élevée,popularité: Faible,popularité: Moyenne,popularité: Élevée,tempo: Lent,tempo: Moyen,tempo: Rapide,valence: Joyeux,valence: Sombre
0,False,True,False,False,True,False,False,True,False,False,...,True,False,False,False,True,False,True,False,False,True
1,False,True,False,False,True,False,False,True,False,False,...,True,False,False,False,True,False,False,True,True,False
2,False,True,False,True,False,False,True,False,False,False,...,True,False,False,True,False,False,False,True,True,False
3,False,True,False,False,True,False,True,False,False,False,...,False,False,False,True,False,False,False,True,False,True
4,False,True,False,True,False,False,False,True,False,False,...,True,False,False,True,False,False,False,True,True,False


In [13]:
df_train.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(62317, 26)

In [14]:
# Pour appliquer l'algorithme Apriori sur df_train pour générer des itemsets fréquents,
# avec un seuil de support minimum de 0.01 et en utilisant les noms de colonnes.
frequent_itemsets_train = apriori(df_train, min_support=0.02, use_colnames=True)

# Pour afficher les itemsets fréquents générés.
frequent_itemsets_train

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.338559,(acousticness: Calme)
1,0.661441,(acousticness: Énergique)
2,0.149799,(danceability: Faible)
3,0.518590,(danceability: Moyenne)
4,0.331611,(danceability: Élevée)
...,...,...
2872,0.020685,"(energy: Faible, language: English, loudness: ..."
2873,0.020941,"(danceability: Moyenne, valence: Joyeux, tempo..."
2874,0.024295,"(valence: Joyeux, tempo: Rapide, danceability:..."
2875,0.029029,"(energy: Faible, language: English, tempo: Len..."


In [15]:
# Pour générer des règles d'association à partir des itemsets fréquents,
# en utilisant la métrique de 'confidence' avec un seuil minimum de 0.2.
# On utilise également la longueur de df_train pour le nombre d'itemsets.
training_rules = association_rules(frequent_itemsets_train, metric="confidence", min_threshold=0.2,num_itemsets=len(df_train))

# Pour trier les règles d'association par 'confidence' et 'lift' en ordre décroissant,
# et sélectionner les 7 premières colonnes du DataFrame résultant.
training_rules.sort_values(by=['confidence','lift'],ascending=False).iloc[:,:7]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
25683,"(energy: Faible, language: English, tempo: Len...",(valence: Sombre),0.023284,0.490364,0.023252,0.998622,2.036491
26376,"(energy: Faible, language: English, tempo: Len...",(valence: Sombre),0.022931,0.490364,0.022899,0.998600,2.036448
21193,"(energy: Faible, language: English, tempo: Len...",(valence: Sombre),0.030313,0.490364,0.030265,0.998412,2.036064
25573,"(energy: Faible, language: English, tempo: Len...",(valence: Sombre),0.029510,0.490364,0.029462,0.998369,2.035976
24877,"(energy: Faible, language: English, tempo: Len...",(valence: Sombre),0.029446,0.490364,0.029398,0.998365,2.035968
...,...,...,...,...,...,...,...
7038,(language: Korean),"(popularité: Moyenne, loudness: Élevée, acoust...",0.110612,0.044980,0.022129,0.200058,4.447740
4671,"(loudness: Faible, acousticness: Énergique)","(danceability: Faible, language: English)",0.116148,0.100229,0.023236,0.200055,1.995972
16084,"(loudness: Élevée, acousticness: Énergique)","(valence: Joyeux, popularité: Faible, danceabi...",0.177993,0.179181,0.035608,0.200054,1.116494
7129,"(acousticness: Énergique, language: Tamil)","(loudness: Moyenne, valence: Sombre)",0.120401,0.224176,0.024087,0.200053,0.892392


In [16]:
# Pour filtrer les règles d'association où les 'consequents' contiennent "popularité: Élevée",
# puis trier les résultats par 'confidence' et 'lift' en ordre décroissant,
# et enfin sélectionner les 7 premières colonnes du DataFrame résultant.
training_rules[training_rules['consequents'].astype(str).str.contains("popularité: Élevée")].sort_values(by=['confidence','lift'],ascending=False).iloc[:,:7]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
7040,"(language: Korean, loudness: Élevée, acousticn...",(popularité: Élevée),0.061893,0.132388,0.022562,0.364532,2.753520
2902,"(language: Korean, loudness: Élevée)",(popularité: Élevée),0.063466,0.132388,0.022883,0.360556,2.723489
16894,"(language: Korean, loudness: Élevée, acousticn...",(popularité: Élevée),0.056758,0.132388,0.020219,0.356234,2.690841
7043,"(language: Korean, loudness: Élevée)","(popularité: Élevée, acousticness: Énergique)",0.063466,0.103262,0.022562,0.355499,3.442681
10707,"(language: Korean, loudness: Élevée, energy: É...",(popularité: Élevée),0.057625,0.132388,0.020364,0.353383,2.669309
16901,"(language: Korean, loudness: Élevée, energy: É...","(popularité: Élevée, acousticness: Énergique)",0.057625,0.103262,0.020219,0.350877,3.397920
16902,"(language: Korean, loudness: Élevée, acousticn...","(energy: Élevée, popularité: Élevée)",0.061893,0.069179,0.020219,0.326679,4.722255
10710,"(language: Korean, loudness: Élevée)","(energy: Élevée, popularité: Élevée)",0.063466,0.069179,0.020364,0.320860,4.638138
16907,"(language: Korean, loudness: Élevée)","(energy: Élevée, popularité: Élevée, acousticn...",0.063466,0.065584,0.020219,0.318584,4.857647
6412,"(language: Korean, energy: Élevée, acousticnes...",(popularité: Élevée),0.076191,0.132388,0.024263,0.318450,2.405435


In [17]:
# Pour filtrer les règles d'association où les 'consequents' contiennent "popularité: Faible",
# puis trier les résultats par 'confidence' et 'lift' en ordre décroissant,
# et enfin sélectionner les 7 premières colonnes du DataFrame résultant.
training_rules[training_rules['consequents'].astype(str).str.contains("popularité: Faible")].sort_values(by=['confidence','lift'],ascending=False).iloc[:,:7]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
9789,"(tempo: Rapide, loudness: Faible, danceability...",(popularité: Faible),0.027954,0.678114,0.023653,0.846154,1.247806
13090,"(energy: Moyenne, valence: Joyeux, language: T...",(popularité: Faible),0.033025,0.678114,0.027858,0.843537,1.243947
13334,"(valence: Joyeux, language: Tamil, loudness: F...",(popularité: Faible),0.024616,0.678114,0.020669,0.839635,1.238192
11251,"(valence: Joyeux, loudness: Faible, language: ...",(popularité: Faible),0.041000,0.678114,0.034292,0.836399,1.233421
10254,"(energy: Moyenne, loudness: Faible, language: ...",(popularité: Faible),0.029992,0.678114,0.025081,0.836276,1.233239
...,...,...,...,...,...,...,...
16312,"(energy: Moyenne, valence: Joyeux, acousticnes...","(popularité: Faible, language: English)",0.118154,0.261133,0.023669,0.200326,0.767143
24329,"(valence: Joyeux, acousticness: Énergique, lan...","(loudness: Élevée, popularité: Faible, energy:...",0.109521,0.087344,0.021936,0.200293,2.293158
5870,"(danceability: Élevée, acousticness: Énergique)","(popularité: Faible, tempo: Moyen)",0.268161,0.153104,0.053677,0.200168,1.307394
2509,(energy: Élevée),"(popularité: Faible, language: English)",0.407305,0.261133,0.081519,0.200142,0.766438


Réalisé par :  

*   BENDAHAN Oumaima
*   MANSARE Nafissatou
*   ASSOGBA Almoctar